# CAPÍTULO 5. PROCESAMIENTO DE LOS DATOS - PARTE 3 (MACHINE LEARNING)

### Todo el código expuesto forma parte del TFG: "Diseño basado en Big Data para aplicación en ingeniería", presentado en la ETSIDI de la Universidad Politécnica de Madrid, en junio de 2020.   

#### Ejemplo red neuronal para puerta XOR 

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense

In [ ]:
#Se carga el set de datos para el entrenamiento. En este caso, las 4 combinaciones posibles de una entrada XOR.
training_data= np.array([[0,0],[0,1],[1,0],[1,1]], "float32")

#Se carga la salida correcta de la puerta XOR
target_data = np.array([[0],[1],[1],[0]]), "float32")

In [ ]:
#Se construye el modelo

model = Sequential() #Se genera el contenedor de la red neuronal
model.add(Dense(16, input_dim=2, activation='relu')) #16 indica el número de neuronas seleccionadas
model.add(Dense(1, activation='sigmoid')) #Se define la función de activación, en este caso, sigmoide

In [ ]:
#Se entrena al modelo, en este caso aplicando como función de error el error cuadrático medio
model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['binary_accuracy'])

model.fit(training_data, target_data, epochs=1000) #el número de epochs define el número de iteraciones que se realizan

In [ ]:
# se evalua el modelo 
scores = model.evaluate(training_data, target_data)
 
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
print (model.predict(training_data).round())

#### Redes LSTM 

In [ ]:
import numpy as np
np.random.seed(4)
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tools.eval_measures import rmse
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [ ]:
#se definen los datos en variables
df = df['Columna seleccionada']
df = pd.DataFrame(df)

In [ ]:
#Se define el set de entrenamiento y de validacion indicando que fechas limitan cada uno. 
set_entrenamiento = df[:'2018-12-01']
set_validacion = df['2018-12-01':]
set_entrenamiento

In [ ]:
#Se define el escalador que transformará los datos a valores entre 0 y 1. 
scaler = MinMaxScaler()
scaler.fit(set_entrenamiento)
set_entrenamiento = scaler.transform(set_entrenamiento)
set_validacion = scaler.transform(set_validacion)

In [ ]:
n_input = 12 #numero de datos de entrenamiento por cada dato de validación
n_features = 1

generator = TimeseriesGenerator(set_entrenamiento,set_entrenamiento,length=n_input,batch_size=6)

model = Sequential() #contenedor de la red
model.add(LSTM(300,activation='relu',input_shape=(n_input,n_features))) #300 indica el número de neuronas de la red LSTM
model.add(Dropout(0.15))
model.add(Dense(1)) #capa de salida
model.compile(optimizer='adam',loss='mse') #entrenamiento metodo rmsprop = gradiente descendiente

model.fit_generator(generator,epochs=1000) #funcion de error: error cuadratico medio. Lotes de 1000 ejemplo en este caso. 

In [ ]:
#Una vez entrenada la red, se pasa a realizar el test de validación, para comprobar el error de la misma. 

pred_list = set_validacion
pred_list = scaler.transform(pred_list)

pred_list = []

batch = set_entrenamiento[-n_input:].reshape((1,n_input,n_features)) 

for i in range(n_input):
    pred_list.append(model.predict(batch)[0]) #normalizacion inversa de los datos, escala real
    batch = np.append(batch[:,1:,:], [[pred_list[i]]],axis=1)
    

In [ ]:
#Des-escalamos los valores

df_predict = pd.DataFrame(scaler.inverse_transform(pred_list),index=df[-n_input:].index, columns=['Predicción'])

df_test = pd.concat([df,df_predict],axis=1)

In [ ]:
df_test.tail(12) #muestra los valores predecidos por pantalla

In [ ]:
#Visualizamos los resultados del entrenamiento en forma gráfica

plt.figure(figsize=(20,5))
plt.plot(df_test.index,df_test['nombre columna seleccionada'])
plt.plot(df_test.index,df_test['Predicción'],color='r')


In [ ]:
#Una vez entrenado el modelo, creamos lo valores futuros

set_entrenamiento = df

scaler.fit(set_entrenamiento)
set_entrenamiento = scaler.transform(set_entrenamiento)

n_input = 12
n_features = 1

generator = TimeseriesGenerator(set_entrenamiento,set_entrenamiento,length=n_input,batch_size=6)

model.fit_generator(generator,epochs=1000)

pred_list = []

batch = set_entrenamiento[-n_input:].reshape((1,n_input,n_features))

for i in range(n_input):
    pred_list.append(model.predict(batch)[0])
    batch = np.append(batch[:,1:,:], [[pred_list[i]]],axis=1)

In [ ]:
#Los añadimos a una nueva columna de nuestro Dataframe para que queden guardados

from pandas.tseries.offsets import DateOffset
add_dates = [df.index[-1] + DateOffset(months=x) for x in range(0,13)]
future_dates = pd.DataFrame(index=add_dates[1:],columns=df.columns)

In [ ]:
#Des-escalamos los resultados

df_predict = pd.DataFrame(scaler.inverse_transform(pred_list),
                                 index=future_dates[-n_input:].index,columns=['Predicciones'])

df_proj = pd.concat([df,df_predict],axis=1)

In [ ]:
#Visualizamos los resultados de forma gráfica

plt.figure(figsize=(10,4))
plt.plot(df_proj.index,df_proj['nombre columna seleccionada'])
plt.plot(df_proj.index,df_proj['Predicciones'],color='r')
plt.legend(loc='best',fontsize='large')
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show

#### Todo el código expuesto forma parte del TFG: "Diseño basado en Big Data para aplicación en ingeniería", presentado en la ETSIDI de la Universidad Politécnica de Madrid, en junio de 2020.    